In [64]:
import spotipy
import spotipy.util as util
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind




In [2]:
%env SPOTIPY_CLIENT_ID = "16a4e0ff93334a0ba81f6f784e0292b7"
%env SPOTIPY_CLIENT_SECRET = '7acb5218466741a18896ac54797a0156'

scope = 'playlist-read-private playlist-read-collaborative user-library-read user-read-recently-played user-top-read'
redirect_uri = "http://localhost/"
username = "chriscc301916"
client_id = "16a4e0ff93334a0ba81f6f784e0292b7"
client_secret = "7acb5218466741a18896ac54797a0156"


post_uri = 'spotify:artist:246dkjvS1zLTtiykXe5h60'

spotify = spotipy.client.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret))

results = spotify.artist_albums(post_uri, album_type='album')
albums = results['items']

while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

env: SPOTIPY_CLIENT_ID="16a4e0ff93334a0ba81f6f784e0292b7"
env: SPOTIPY_CLIENT_SECRET='7acb5218466741a18896ac54797a0156'


In [3]:
album_uris = [albums[i]["uri"] for i in range(len(albums))]

pm_songs  = {}
for uri in album_uris:
    album = spotify.album_tracks(uri)['items']
    for i in range(len(album)):
        pm_songs[album[i]['name']] = album[i]['uri']
print(pm_songs)


{"Hollywood's Bleeding": 'spotify:track:19iGLq3w4Q1e8gg6qSGfRj', 'Saint-Tropez': 'spotify:track:51fG7ydQm27FiPo88Gi1RT', 'Enemies (feat. DaBaby)': 'spotify:track:70kYmAusgHwkBIk944STcj', 'Allergic': 'spotify:track:0mDmG70LDwF8pLDRQ2UgT7', 'A Thousand Bad Times': 'spotify:track:2IlaU3eLHt3NpylRdBmi7Z', 'Circles': 'spotify:track:6KjegbJPzNRetrgFJiHVMp', 'Die For Me (feat. Future & Halsey)': 'spotify:track:1AuH8eXCNIzojMqt5xBrSO', 'On The Road (feat. Meek Mill & Lil Baby)': 'spotify:track:0Y8QvQxB3fraIEBuWPW9er', 'Take What You Want (feat. Ozzy Osbourne & Travis Scott)': 'spotify:track:3sORDiSRTAlxecx5xJAqc8', "I'm Gonna Be": 'spotify:track:58tGA0YI9A4125KpIZhkHU', 'Staring At The Sun (feat. SZA)': 'spotify:track:0KEspatN1t1SAZUfm22c4z', 'Sunflower - Spider-Man: Into the Spider-Verse': 'spotify:track:6nY9ga57plhBYQnZX0Oqe8', 'Internet': 'spotify:track:4U3m2QS28oO5VM7G9BiRme', 'Goodbyes (feat. Young Thug)': 'spotify:track:6cIYFBwTv4JGpX7wOk5VIu', 'Myself': 'spotify:track:1Sg6DGutLrmuhzY1G3

In [59]:
data = pd.read_csv("C:\\Users\\chris\\Documents\\GitHub\\Crescendo\\data\\topCharts.csv")

data = data.drop(columns = ["Unnamed: 0", "Id"], axis = 1)
data["Date"] = pd.to_datetime(data["Date"])

pm_topdata = data[data["Artist"] == "Post Malone"]
pm_topdata = pm_topdata.rename(columns = dict(zip(pm_topdata.columns, [col.lower() for col in pm_topdata.columns])))
pm_topdata = pm_topdata.rename(columns = {"duration": "duration_ms", "time signature":"time_signature"})

pm_topdata = pm_topdata.drop(columns = ["artist", "date", "rank", "streams"])

pm_songnames = set(pm_songs.keys())
pm_topsongnames = set(pm_topdata["title"])

pm_nottopsongnames = pm_songnames.difference(pm_topsongnames)


In [60]:
song_uris = [pm_songs[song] for song in pm_nottopsongnames]

song_features_dict = {}
song_features = spotify.audio_features(tracks = song_uris)
for i in song_features[0].keys():
    song_features_dict[i] = []

for i in range(len(song_features)):
    sf = song_features[i]
    for k in sf.keys():
        song_features_dict[k].append(sf[k])


pm_nottopdata = pd.DataFrame.from_dict(song_features_dict)
pm_nottopdata.insert(loc = 0, column = "title", value = pm_nottopsongnames)
pm_nottopdata = pm_nottopdata.drop(columns = ["analysis_url", "track_href", "uri", "id", "type"])
# print(pm_nottopdata)



In [61]:
col_order = list(pm_topdata.columns)

col_order.extend(list(set(pm_nottopdata.columns).difference(set(pm_topdata.columns))))
print(col_order)
pm_nottopdata = pm_nottopdata[col_order]

print(pm_topdata.columns)
print(pm_nottopdata.columns)

['title', 'duration_ms', 'time_signature', 'tempo', 'key', 'mode', 'valence', 'danceability', 'energy', 'acousticness', 'instrumentalness', 'liveness', 'speechiness', 'loudness']
Index(['title', 'duration_ms', 'time_signature', 'tempo', 'key', 'mode',
       'valence', 'danceability', 'energy', 'acousticness',
       'instrumentalness'],
      dtype='object')
Index(['title', 'duration_ms', 'time_signature', 'tempo', 'key', 'mode',
       'valence', 'danceability', 'energy', 'acousticness', 'instrumentalness',
       'liveness', 'speechiness', 'loudness'],
      dtype='object')


In [69]:
print(pm_topdata.mean())
print()
print(pm_nottopdata.mean())


duration_ms         207180.243552
time_signature           3.995784
tempo                  132.830598
key                      5.767237
mode                     0.646329
valence                  0.379595
danceability             0.647474
energy                   0.610406
acousticness             0.287563
instrumentalness         0.002217
dtype: float64

duration_ms         224629.058824
time_signature           4.000000
tempo                  122.805529
key                      4.823529
mode                     0.588235
valence                  0.323400
danceability             0.621235
energy                   0.568706
acousticness             0.414118
instrumentalness         0.000308
liveness                 0.119159
speechiness              0.087547
loudness                -6.676000
dtype: float64


title                object
duration_ms         float64
time_signature        int64
tempo               float64
key                   int64
mode                  int64
valence             float64
danceability        float64
energy              float64
acousticness        float64
instrumentalness    float64
dtype: object

In [72]:
for var in pm_topdata.columns:
    
    #ho: top[var] == nottop[var]
    #ha: top[var] != nottop[var]
    if pm_topdata[var].dtypes != "object":
        print(f"{var}: {ttest_ind(pm_topdata[var], pm_nottopdata[var], equal_var = False)}")

duration_ms: Ttest_indResult(statistic=-2.0001395952831342, pvalue=0.06269976208839874)
time_signature: Ttest_indResult(statistic=-1.4046117807534837, pvalue=0.160175272310743)
tempo: Ttest_indResult(statistic=1.4834758941665267, pvalue=0.15729709303138356)
key: Ttest_indResult(statistic=1.3713664042918061, pvalue=0.1890916656652412)
mode: Ttest_indResult(statistic=0.4717212328842987, pvalue=0.6434709411056503)
valence: Ttest_indResult(statistic=1.4836678860018229, pvalue=0.15720151279800487)
danceability: Ttest_indResult(statistic=0.9063153854305085, pvalue=0.3781736521018636)
energy: Ttest_indResult(statistic=1.321937685827049, pvalue=0.20472195407087618)
acousticness: Ttest_indResult(statistic=-3.1029384703158343, pvalue=0.006800498942874899)
instrumentalness: Ttest_indResult(statistic=3.8365799313648554, pvalue=0.0001842954947498194)
